In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict
sp=spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id='',client_secret=''))
def find_song(name,year):
    song_data=defaultdict()
    results=sp.search(q='track:{} year:{}'.format(name,year),limit=1)
    if results['tracks']['items']==[]:
        return None
    results=results['tracks']['items'][0]
    track_id=results['id']
    audio_features=sp.audio_features(track_id)[0]
    song_data['name']=[name]
    song_data['year']=[year]
    song_data['explicit']=[int(results['explicit'])]
    song_data['duration_ms']=[results['duration_ms']]
    song_data['popularity']=[results['popularity']]
    for key,value in audio_features.items():
        song_data[key]=value
    song_data_df=pd.DataFrame(song_data)
    song_cluster_label=song_cluster_pipeline.predict(song_data_df[number_cols])
    song_data_df['cluster_label']=song_cluster
    return song_data_df

In [7]:
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
import difflib
def get_song_data(song):
    try:
        song_data=data[(data['name']==song['name'])&(data['year']==song['year'])].iloc[0]
        return song_data
    except IndexError:
        return find_song(song['name'],song['year'])
def get_mean_vector(song_list):
    song_vectors=[]
    for song in song_list:
        song_data=get_song_data(song)
        if song_data is None:
            continue
        song_vector=song_data[number_cols].values
        song_vectors.append(song_vector)
        song_matrix=np.array(list(song_vectors))
        return np.mean(song_matrix,axis=0)
def flatten_dict_list(dict_list):
    flattened_dict=defaultdict()
    for key in dict_list[0].keys():
        flattened_dict[key]=[]
    for dictionary in dict_list:
        for key,value in dictionary.items():
            flattened_dict[key].append(value)
    return flattened_dict

In [9]:
def recommender(song_list):
    rec_song_list_column=['name','year','artists']
    song_dict=flatten_dict_list(song_list)
    song_center=get_mean_vector(song_list)
    scaler=song_cluster_pipeline.steps[0][1]
    scaled_data=scaler.transform(data[number_cols])
    scaled_song_center=scaler.transform(song_center.reshape(1,-1))
    distance=cdist(scaled_song_center,scaled_data,'cosine')
    index=list(np.argsort(distance)[:,:10][0])
    rec_songs=data.ilot[index]
    rec_songs=rec_songs[-rec_songs['name'].isin(song_dict['name'])]
    return rec_songs[rec_song_list_column].to_dict(orient='records')

In [12]:
#   recommender([{'name':'Animals','year':2013}])

In [14]:
# recommender([{'name':'Story of My Life','year':2013}])